In [1]:
sc.applicationId

u'application_1560417165231_26225'

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
from pyspark.ml.feature import Imputer
import re
from matplotlib.backends.backend_pdf import PdfPages
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [6]:
select_var=['partygenid', 'disbursement_month', 'poor_payment_flag', 'default', 'duration_poor_payment_count', 
            'duration_poor_payment_distinct', 'max_severity_class', 'mean_severity_class', 'n_card_trx_all', 
            'amt_card_trx_all', 'n_card_trx_f', 'amt_card_trx_f', 'n_currency', 'age', 
            'NumOfTransNight_last_6_months', 'NumOfTransDay_last_6_months', 
            'AmountOfTransNight_last_6_months', 'AmountOfTransDay_last_6_months', 
            'income_amt_6_std', 'income_amt_6_avg', 
            'nr_totalproducts', 'nr_savingsInsuranceproducts', 'nr_Lendingproducts', 'nr_Paymentsproducts',
            'NumOfPlannedTransfersTotalMonth_avg_last_6m', 'AmountOfTotalMonthlyPlannedTransfers_avg_last_6m',
            'TotalNumOfDepositsMonth_avg_last_6m', 'NumOfATMDepositsMonth_avg_last_6m', 'AmountOfATMDepositsMonth_avg_last_6m', 
            'AmountOfTotalWithdrawalsMonth_avg_last_6m', 'NumOfATMWithdrawalsMonth_avg_last_6m', 
            'AmountOfATMWithdrawalsMonth_avg_last_6m', 'NumOfTransactionsActiveCustomer_avg_last_6m', 
            'ConsumerLoansCompetitors_flag_last_6_months']

In [7]:
df_base_v2 = spark.table('ddp_consumerlending_vault.p901cyo_combined_v1')
df_base = df_base_v2.select(select_var)
# cache df_base
print(df_base.cache().count())

345852


In [8]:
# one way to fill the missings--> all with 0
df_base_fillna = df_base.fillna(0)

In [9]:
drop_feature=['default', 'poor_payment_flag', 'partygenid', 'disbursement_month']
feature_grp=[x for x in df_base.columns if x not in drop_feature]

feature_numeric = [col for col in feature_grp if (df_base.select(col).dropna().distinct().count()>10)]
print(len(feature_numeric),feature_numeric)

(28, ['duration_poor_payment_count', 'duration_poor_payment_distinct', 'mean_severity_class', 'n_card_trx_all', 'amt_card_trx_all', 'n_card_trx_f', 'amt_card_trx_f', 'n_currency', 'age', 'NumOfTransNight_last_6_months', 'NumOfTransDay_last_6_months', 'AmountOfTransNight_last_6_months', 'AmountOfTransDay_last_6_months', 'income_amt_6_std', 'income_amt_6_avg', 'nr_totalproducts', 'nr_savingsInsuranceproducts', 'nr_Lendingproducts', 'nr_Paymentsproducts', 'NumOfPlannedTransfersTotalMonth_avg_last_6m', 'AmountOfTotalMonthlyPlannedTransfers_avg_last_6m', 'TotalNumOfDepositsMonth_avg_last_6m', 'NumOfATMDepositsMonth_avg_last_6m', 'AmountOfATMDepositsMonth_avg_last_6m', 'AmountOfTotalWithdrawalsMonth_avg_last_6m', 'NumOfATMWithdrawalsMonth_avg_last_6m', 'AmountOfATMWithdrawalsMonth_avg_last_6m', 'NumOfTransactionsActiveCustomer_avg_last_6m'])


In [271]:
# another way to fill the missings -->medium
df_base_cast = df_base.select('default', 'poor_payment_flag', 'partygenid', 'disbursement_month', *(F.col(c).cast("float").alias(c) for c in feature_grp))
#df_base_cast = df_base.select( *(F.col(c).cast("float").alias(c) for c in feature_grp))
feature_numeric_out = [s + '_out' for s in feature_numeric]
imputer = Imputer(inputCols=feature_numeric, outputCols=feature_numeric)
df_base_fillmedian = imputer.setStrategy("median").fit(df_base_cast).transform(df_base_cast)
df_base_fillm = df_base_fillmedian.fillna(0)

In [272]:
# get bin boundary from decision tree
def optimal_binning_boundary(df, var):
    
    from sklearn.tree import DecisionTreeClassifier

    boundary = []  # return the boundaries
    
    x = df.select(var).toPandas()  # send to pandas
    y = df.select('default').toPandas()
    
    clf = DecisionTreeClassifier(criterion='entropy',   
                                 max_leaf_nodes=6,       
                                 min_samples_leaf=0.05)

    clf.fit(x, y)  
    
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    threshold = clf.tree_.threshold
    
    for i in range(n_nodes):
        if children_left[i] != children_right[i]:  
            boundary.append(threshold[i])

    min_x = x.min()
    max_x = x.max() + 0.1  # +0.1 is to make sure after groupby the maximum sample value will be included
    boundary.append(min_x[0])
    boundary.append(max_x[0])
    boundary.sort()
    

    return boundary

In [273]:
# plotting function
def bin_plot(data, var):
    plt.clf
    fig = plt.figure()
    plt.xticks(data['cell'], figure=fig)
    plt.ylabel('default_rate', figure=fig)
    plt.title('bin of %s vs default rate' %var, figure=fig)
    plt.bar(data['cell'].astype('float'),data['bad_rate'], figure=fig)
    #plt.show()
    plt.close()
    return fig

In [274]:
def bin_iv_var2(method, df, var, cell):
    if method == 'dt':
        df_nomiss = df.select(var,'default').dropna()
        df_miss = df.select(var,'default').filter(F.col(var).isNull()) 
        x = df_nomiss.select(var).toPandas()  # send to pandas
        y = df_nomiss.select('default').toPandas()
        boundary = optimal_binning_boundary(df=df_nomiss, var=var)       
        df_pd = pd.concat([x, y], axis=1)                        
        df_pd.columns = ['x', 'y']                               
        df_pd['bins'] = pd.cut(x=x[var], bins=boundary, right=False)  
        if len(df_miss.head(1))>0:
            df_miss_pd = df_miss.toPandas()
            df_miss_pd.columns = ['x','y']
            df_miss_pd['bins'] = 'missing'
            df_pd = df_pd.append(df_miss_pd)

        grouped = df_pd.groupby('bins')['y']                     
        result_df = grouped.agg([('good',  lambda y: (y == 0).sum()), 
                                 ('bad',   lambda y: (y == 1).sum()),
                                 ('total', 'count')])
    elif method == 'other':
        df_pd = df.select('default',var).toPandas()
        df_pd['bins'] = pd.qcut(df_pd[var].astype('float'),cell,labels=False)
        df_pd.fillna(-1,inplace=True)
        grouped1 = df_pd.groupby('bins')['default']
        result_df1 = grouped1.agg([('good',  lambda default: (default == 0).sum()), 
                                 ('bad',   lambda default: (default == 1).sum()),
                                 ('total', 'count')])
        grouped2 = df_pd.groupby('bins')[var]
        result_df2 = grouped2.agg([('max',  lambda var: (max(var)+0.1)), 
                                 ('min',   lambda var: (min(var)))])
        result_df = pd.concat([result_df1, result_df2], axis=1, sort=False)

    result_df['var'] = var
    result_df['cell'] = np.arange(start=1,stop=result_df.shape[0]+1,step=1)
    result_df['bins'] = result_df.index
    result_df['good_pct'] = result_df['good'] / result_df['good'].sum()       
    result_df['bad_pct'] = result_df['bad'] / result_df['bad'].sum()          
    result_df['total_pct'] = result_df['total'] / result_df['total'].sum()    

    result_df['bad_rate'] = result_df['bad'] / result_df['total']             
    
    result_df['woe'] = np.log(result_df['good_pct'] / result_df['bad_pct'])              
    result_df['iv'] = (result_df['good_pct'] - result_df['bad_pct']) * result_df['woe'] 

    return result_df

In [275]:
# decision tree method
for segment in range(1,2):
    for basedata in (df_base, df_base_fillna, df_base_fillm):
        # filter the dataset based on the segment
        df=basedata.filter(F.col('poor_payment_flag')==segment)
        print(df.cache().count())
        df.groupby('default').count().show() 
        # feature list
        drop_feature=['default', 'poor_payment_flag', 'partygenid', 'disbursement_month','gender','check','perf_n']
        feature_grp=[x for x in df.columns if x not in drop_feature]
        # calculate information value
        # define output variables needed
        combine_result = pd.DataFrame()
        # output graphs and values
        if basedata == df_base:
            dataname = str('nofill')
        elif basedata == df_base_fillna:
            dataname = str('fillna')
        elif basedata == df_base_fillm:
            dataname = str('fillm')
            
        pdfname = 'Charts_'+str(dataname)+str(segment)+'.pdf'
        with PdfPages(pdfname) as export_pdf:
            for i in range(0,len(feature_grp)): 
                result_df = bin_iv_var2(method='dt', df=df, var=feature_grp[i], cell=10) 
                combine_result = combine_result.append(result_df, ignore_index = True)
                fig = bin_plot(data=result_df, var=var)
                export_pdf.savefig(figure=fig)       
            binfeaturecsv = "feature_bin_"+str(dataname)+str(segment)+".csv"
            combine_result.to_csv(binfeaturecsv, sep=',', encoding='utf-8')
        print(str(segment), dataname, len(feature_grp))

102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'nofill', 30)
102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'fillna', 30)
102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'fillm', 30)


In [200]:
# logistic regression method (under development, can't be used)
for segment in range(1,2):
#     for basedata in (df_base, df_base_fillna, df_base_fillm):
    for basedata in (df_base, df_base_fillna):
        # filter the dataset based on the segment
        df=basedata.filter(F.col('poor_payment_flag')==segment)
        print(df.cache().count())
        df.groupby('default').count().show() 
        
        if basedata == df_base:
            dataname = str('nofill')
        elif basedata == df_base_fillna:
            dataname = str('fillna')
        elif basedata == df_base_fillm:
            dataname = str('fillm')

        combine_result = pd.DataFrame()
        for i in range(0,len(feature_numeric)): 
            df_pd = df.select('default',feature_numeric[i]).toPandas()
            n = len(df_pd['default'])
            default_n = sum(df_pd['default'])
            nondefault_n = len(df_pd['default']) - sum(df_pd['default'])
            var = feature_numeric[i]

            cell_n = 10

            while cell_n>=1:
                try:
                    cell_n = cell_n
                    result_df = bin_iv_var2(method='other', df=df, var=var, cell=cell_n)
                    break
                except ValueError:
                    cell_n = cell_n-1
                    continue
                except TypeError:
                    print('%s something is wrong with the graph' %var)
                    break
                except KeyboardInterrupt:
                    print('I want to stop this')
                    break
            combine_result = combine_result.append(result_df, ignore_index=False) 
            binfeaturecsv = "feature_bin_other_"+str(dataname)+str(segment)+".csv"
            combine_result.to_csv(binfeaturecsv, sep=',', encoding='utf-8')
        print(str(segment), dataname, len(feature_numeric))

102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'nofill', 28)
102239
+-------+------+
|default| count|
+-------+------+
|      1|   689|
|      0|101550|
+-------+------+

('1', 'fillna', 28)


In [205]:
df_pd.columns[1]

'NumOfTransactionsActiveCustomer_avg_last_6m'

In [206]:
combine_result[combine_result['var']==df_pd.columns[1]].head(5)

,good,bad,total,max,min,var,cell,bins,good_pct,bad_pct,total_pct,bad_rate,woe,iv
bins,,,,,,,,,,,,,,
0,10094,136,10230,30.600000,0.000000,NumOfTransactionsActiveCustomer_avg_last_6m,1,0,0.099399,0.197388,0.100060,0.013294,-0.686024,0.067222
1,10257,80,10337,50.266667,30.666667,NumOfTransactionsActiveCustomer_avg_last_6m,2,1,0.101004,0.116110,0.101106,0.007739,-0.139376,0.002105
2,10035,72,10107,61.433333,50.200000,NumOfTransactionsActiveCustomer_avg_last_6m,3,2,0.098818,0.104499,0.098857,0.007124,-0.055897,0.000318
3,10331,51,10382,70.933333,61.500000,NumOfTransactionsActiveCustomer_avg_last_6m,4,3,0.101733,0.074020,0.101546,0.004912,0.318013,0.008813
4,10124,52,10176,80.100000,71.000000,NumOfTransactionsActiveCustomer_avg_last_6m,5,4,0.099695,0.075472,0.099531,0.005110,0.278355,0.006743


In [276]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html
df_pd = df.select('default',var).toPandas()
df_pd['bins'] = pd.qcut(df_pd[var].astype('float'),10,labels=False)
df_pd.fillna(-1,inplace=True)
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categorical_features = [1])
df_array = onehot.fit_transform(df_pd.drop([var],axis=1)).toarray()

In [309]:
from sklearn.feature_selection import RFE
#from sklearn.svm import SVR
#estimator = SVR(kernel="linear")
#selector = RFE(estimator, 5, step=1)
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression()
selector = RFE(estimator, 5, step=1)
selector = selector.fit(x, y)
selector.support_

array([False, False, False, False,  True,  True,  True,  True,  True, False], dtype=bool)

In [310]:
selector.ranking_

array([6, 2, 3, 4, 1, 1, 1, 1, 1, 5])

In [294]:
x = df_array[:,0:(df_array.shape[1]-1)]
y = df_array[:,-1]

In [302]:
group = df_pd.groupby(['bins']).count()

In [303]:
group['ranking'] = selector.ranking_

In [304]:
group

,default,NumOfTransactionsActiveCustomer_avg_last_6m,ranking
bins,,,
0,10252,10252,6
1,10234,10234,5
2,10375,10375,4
3,15184,15184,3
4,5187,5187,2
5,10192,10192,1
6,10216,10216,1
7,10256,10256,1
8,10137,10137,1


In [313]:
from sklearn.feature_selection import SelectKBest, f_regression
feat_selector = SelectKBest(f_regression, k=5)
best = feat_selector.fit(x, y)

In [315]:
best.

array([  4.37082694e-13,   4.47189571e-01,   7.11808395e-01,
         2.09549145e-01,   1.50607587e-02,   1.72766768e-01,
         7.75718501e-02,   4.02982058e-02,   6.71205291e-02,
         2.39675840e-01])